# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(42)
initial_params = np.random.random([31])

INITIALIZATION_METHOD = 'Amplitude'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 2, 4, 6, 8]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=3)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(3), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(3), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[0, 1])

    # Second layer
    ArbitraryUnitary(params[15:30], wires=[1, 2])

    # Third layer
    qml.RY(params[30], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485]

Example features: [-1.9908578   0.59322636 -1.19287186 -0.53177743 -1.54502451 -1.40664986
  0.18768066  0.84348461]

Expectation value: 0.11885670143193017

 0: ──╭QubitStateVector(M0)──H──RZ(0.375)──H──RX(1.57)──RZ(0.951)──RX(-1.57)──RZ(0.732)──╭RZ(0.599)──H──────────────────────╭RZ(0.156)──H──RX(1.57)──╭RZ(0.0581)──RX(-1.57)──RX(1.57)──╭RZ(0.866)──RX(-1.57)────────────╭RZ(0.601)──H────────────────────────────────────────────────────╭RZ(0.0206)──H──────────H──╭RZ(0.97)──H──RX(1.57)──╭RZ(0.832)──RX(-1.57)──╭RZ(0.212)─────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.497719087635054
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.5418967587034814
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.6384153661464587
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.6734693877551021
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.6765906362545018
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.6914765906362546
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.6857142857142857
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.6876350540216086
epoch 71

epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.6660264105642257
test_accuracies: [0.6857142857142857, 0.6876350540216086, 0.7068427370948379, 0.6794717887154862, 0.7030012004801921, 0.7020408163265306, 0.6744297719087635, 0.6854741896758704, 0.6859543817527011, 0.6869147659063626, 0.6775510204081633, 0.7102040816326531, 0.6974789915966386, 0.6921968787515006, 0.7063625450180072, 0.6837935174069627, 0.6866746698679471, 0.672749099639856, 0.6921968787515006, 0.6938775510204082, 0.6965186074429772, 0.7015606242496999, 0.7001200480192077, 0.707563025210084, 0.6797118847539015, 0.6986794717887155, 0.7015606242496999, 0.7025210084033613, 0.668187274909964, 0.6660264105642257]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.6890756302521008
test_accuracies: [0.6876350540216086, 0.7068427370948379, 0.6794717887154862, 0.7030012004801921, 0.7020408163265306, 0.6744297719087635, 0.6854741

epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.694357743097239
test_accuracies: [0.7102040816326531, 0.6974789915966386, 0.6921968787515006, 0.7063625450180072, 0.6837935174069627, 0.6866746698679471, 0.672749099639856, 0.6921968787515006, 0.6938775510204082, 0.6965186074429772, 0.7015606242496999, 0.7001200480192077, 0.707563025210084, 0.6797118847539015, 0.6986794717887155, 0.7015606242496999, 0.7025210084033613, 0.668187274909964, 0.6660264105642257, 0.6890756302521008, 0.702280912364946, 0.6921968787515006, 0.6770708283313325, 0.7056422569027612, 0.6914765906362546, 0.6849939975990396, 0.6912364945978391, 0.7058823529411764, 0.6842737094837935, 0.694357743097239]


In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 9.12180257e-02  9.67686499e-01  8.99624057e-01  1.48728675e+00
 -1.50565150e-01  1.78229252e-01  1.59038344e-01  5.81134803e-01
  1.06602625e+00  8.39983805e-01  1.30923904e-01  1.57402072e+00
  1.41114897e+00 -1.32769243e-01 -1.35267731e-01 -1.49606316e-03
  7.21590874e-01  3.50560502e-01  1.09979902e+00  7.61467926e-01
  3.74585339e-01 -3.79917095e-01  1.60436285e+00 -6.54416497e-02
  5.29795469e-01  1.02862046e+00  2.62294645e-01 -9.06106922e-01
  1.53256093e+00  3.52456313e-01  7.97372893e-01]


# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7074285714285714


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 10.711228132247925
training time: 3532.2925877571106
test time: 163.677428483963
total time: 3706.6812443733215
